# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating arguments with your requests. 

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.  

In this chat scenario, as the user talks back and forth with the bot, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. 

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.7.1

Let's define a prompt outlining a dialogue chat bot.

In [2]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

Register your semantic function

In [3]:
var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

Initialize your arguments

In [4]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

Chat with the Bot

In [5]:
var userInput = "Hi, I'm looking for book suggestions";
arguments["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(kernel, arguments);

Update the history with the output and set this as the new input value for the next request

In [6]:
history += $"\nUser: {userInput}\nAI: {bot_answer}\n";
arguments["history"] = history;

Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hello! I'd be happy to help. What genre or type of books are you interested in?



Keep Chatting!

In [7]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [8]:
await Chat("I would like a non-fiction book suggestion about Egyptian history. Please only list one book.");


User: I would like a non-fiction book suggestion about Egyptian history. Please only list one book.
AI: I recommend "The Oxford History of Ancient Egypt" by Ian Shaw. It provides a comprehensive overview of ancient Egyptian history and culture.



In [9]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: Some of the topics covered in the book include the political history of ancient Egypt, the development of Egyptian society and culture, religious beliefs and practices, art and architecture, and the legacy of ancient Egypt.



In [10]:
await Chat("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: I don't know specifically what most people find interesting, but topics such as religious beliefs and practices, art and architecture, and the political history of ancient Egypt are often popular among readers interested in Egyptian history.



In [11]:
await Chat("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Sure! Here are some more books you could consider reading about Egyptian history:

1. "The Complete Gods and Goddesses of Ancient Egypt" by Richard H. Wilkinson
2. "Temples, Tombs, and Hieroglyphs: A Popular History of Ancient Egypt" by Barbara Mertz
3. "The Rise and Fall of Ancient Egypt" by Toby Wilkinson
4. "The Pharaohs" by Joyce Tyldesley
5. "The Egyptian Book of the Dead: The Book of Going Forth by Day" translated by Raymond Faulkner



After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hello! I'd be happy to help. What genre or type of books are you interested in?

User: I would like a non-fiction book suggestion about Egyptian history. Please only list one book.
AI: I recommend "The Oxford History of Ancient Egypt" by Ian Shaw. It provides a comprehensive overview of ancient Egyptian history and culture.

User: that sounds interesting, what are some of the topics I will learn about?
AI: Some of the topics covered in the book include the political history of ancient Egypt, the development of Egyptian society and culture, religious beliefs and practices, art and architecture, and the legacy of ancient Egypt.

User: Which topic from the ones you listed do you think most people find interesting?
AI: I don't know specifically what most people find interesting, but topics such as religious beliefs and practices, art and architecture, and the political history of ancient Egypt are often popular among readers interested in Egy